In [1]:
%matplotlib inline

import pandas as pd
import urllib
import json

In [2]:
# endpoint for eviction notices in SF
endpoint_url = "https://data.sfgov.org/resource/5cei-gny5.json"

# open a connection to the URL
connection = urllib.urlopen(endpoint_url)

# download the results
results = connection.read()

print results[:500]  # first 500 characters
print type(results)

[ {
  "other_cause" : false,
  "failure_to_sign_renewal" : false,
  "demolition" : false,
  "ellis_act_withdrawal" : false,
  "development" : false,
  "state" : "CA",
  "city" : "San Francisco",
  "client_location" : {
    "needs_recoding" : false,
    "longitude" : "-122.444468830184",
    "latitude" : "37.7513305524855",
    "human_address" : "{\"address\":\"\",\"city\":\"\",\"state\":\"\",\"zip\":\"\"}"
  },
  "breach" : true,
  "owner_move_in" : false,
  "access_denial" : false,
  "non_payme
<type 'str'>


In [5]:
# the results are a string with JSON-formatted data inside

# parse the string into a Python data structure
data = json.loads(results)

print type(data)
print len(data)

<type 'list'>
1000


In [6]:
ds = pd.DataFrame(data)

In [7]:
ds.head()

,access_denial,address,breach,capital_improvement,city,client_location,condo_conversion,constraints,constraints_date,demolition,...,non_payment,nuisance,other_cause,owner_move_in,roommate_same_unit,state,substantial_rehab,supervisor_district,unapproved_subtenant,zip
0,False,600 Block of Burnett Avenue,True,False,San Francisco,"{u'latitude': u'37.7513305524855', u'needs_rec...",False,0,NaN,False,...,False,False,False,False,False,CA,False,8,False,94131
1,False,2600 Block of 22nd Street,False,False,San Francisco,"{u'latitude': u'37.7560269818259', u'needs_rec...",False,0,NaN,False,...,False,True,False,False,False,CA,False,9,False,94110
2,False,1000 Block of Chestnut Street,True,False,San Francisco,"{u'latitude': u'37.8028649955598', u'needs_rec...",False,0,NaN,False,...,False,False,False,False,False,CA,False,2,False,94103
3,False,600 Block of Burnett Avenue,True,False,San Francisco,"{u'latitude': u'37.7513305524855', u'needs_rec...",False,0,NaN,False,...,False,False,False,False,False,CA,False,8,False,94131
4,False,1300 Block of kansas Street,False,False,San Francisco,"{u'latitude': u'37.7526024924613', u'needs_rec...",False,0,NaN,True,...,False,False,False,False,False,CA,False,10,False,94107


In [9]:
ds.columns

Index([u'access_denial', u'address', u'breach', u'capital_improvement',
       u'city', u'client_location', u'condo_conversion', u'constraints',
       u'constraints_date', u'demolition', u'development',
       u'ellis_act_withdrawal', u'eviction_id', u'failure_to_sign_renewal',
       u'file_date', u'illegal_use', u'late_payments', u'lead_remediation',
       u'neighborhood', u'non_payment', u'nuisance', u'other_cause',
       u'owner_move_in', u'roommate_same_unit', u'state', u'substantial_rehab',
       u'supervisor_district', u'unapproved_subtenant', u'zip'],
      dtype='object')

In [23]:
ds.client_location[1]['longitude']

u'-122.40804532047'

In [27]:
type(ds.client_location)

pandas.core.series.Series

In [34]:
def getcoord(row, val):
    loc = row['client_location']
    var = 'latitude'
    if (type(loc) == dict) & (var in loc.keys()):
        lat = loc['latitude']
        long = loc['longitude']
        if val == 1:
            return lat
        else:
            return long
    else:
        return None

In [36]:
ds['latitude'] = ds.apply(getcoord, axis =1, args = (1,))
ds['longitude'] = ds.apply(getcoord, axis =1, args = (2,))

In [38]:
print len(ds[ds.latitude.isnull() == 1])

11


In [41]:
ds.latitude = ds.latitude.astype('float64')
ds.longitude = ds.longitude.astype('float64')

In [42]:
df = ds[ds.latitude.isnull() == 0]

In [43]:
print len(df)

989


In [44]:
df.to_csv('A10data.csv')